###Importing the modules

In [0]:
from pyspark.sql.functions import *
from delta.tables import DeltaTable

###Reading data from silver layer

In [0]:
prod_silver_df = spark.read.table("retail_analytics.silver.products")

###Selecting the needed columns and add metadata

In [0]:
dim_prod_df = (
    prod_silver_df
    .select(
        col("product_id"),
        col("category"),
        col("sub_category"),
        col("description_pt"),
        col("description_de"),
        col("description_fr"),
        col("description_es"),
        col("description_en"),
        col("description_zh"),
        col("color"),
        col("sizes"),
        col("production_cost")
    )
    .withColumn("_created_at", current_timestamp())
    .withColumn("_updated_at", current_timestamp())
)

###Creating gold table

In [0]:
spark.sql("""
CREATE TABLE IF NOT EXISTS retail_analytics.gold.dim_products (
    product_sk BIGINT GENERATED ALWAYS AS IDENTITY,
    product_id INT,
    category STRING,
    sub_category STRING,
    description_pt STRING,
    description_de STRING,
    description_fr STRING,
    description_es STRING,
    description_en STRING,
    description_zh STRING,
    color STRING,
    sizes STRING,
    production_cost DOUBLE,
    _created_at TIMESTAMP,
    _updated_at TIMESTAMP
)
USING DELTA
""")

DataFrame[]

###Merge processs(SCD-1)

In [0]:
dim_prod_tbl = DeltaTable.forName(spark, "retail_analytics.gold.dim_products")

(
    dim_prod_tbl.alias("tgt")
    .merge(
        dim_prod_df.alias("src"),
        "tgt.product_id = src.product_id"
    )
    .whenMatchedUpdate(set={
        "category": "src.category",
        "sub_category": "src.sub_category",
        "description_pt": "src.description_pt",
        "description_de": "src.description_de",
        "description_fr": "src.description_fr",
        "description_es": "src.description_es",
        "description_en": "src.description_en",
        "description_zh": "src.description_zh",
        "color": "src.color",
        "sizes": "src.sizes",
        "production_cost": "src.production_cost",
        "_updated_at": "current_timestamp()"
    })
    .whenNotMatchedInsert(values={
        "product_id": "src.product_id",
        "category": "src.category",
        "sub_category": "src.sub_category",
        "description_pt": "src.description_pt",
        "description_de": "src.description_de",
        "description_fr": "src.description_fr",
        "description_es": "src.description_es",
        "description_en": "src.description_en",
        "description_zh": "src.description_zh",
        "color": "src.color",
        "sizes": "src.sizes",
        "production_cost": "src.production_cost",
        "_created_at": "current_timestamp()",
        "_updated_at": "current_timestamp()"
    })
    .execute()
)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.read.table("retail_analytics.gold.dim_products").limit(5).display()

product_sk,product_id,category,sub_category,description_pt,description_de,description_fr,description_es,description_en,description_zh,color,sizes,production_cost,_created_at,_updated_at
1,1021,Feminine,Skirts and Shorts,Saia De Tecido Fluidos Com Cinto,Rock Aus Flüssigkeitsgewebe,Jupe De Tissu Fluide,Falda De Tela Fluida,Skirt Of Fluid Fabric,Sk,Not available,S|M|L|XL,11.8,2026-01-20T07:22:20.888Z,2026-01-20T07:22:20.888Z
2,2737,Feminine,Skirts and Shorts,Saia Curta Com Pregas E Cinto,Kurzer Rock Mit Falten Und Gürtel,Jupe Courte Avec Plis Et Ceinture,Falda Corta Con Pliegues Y Cinturón,Short Skirt With Folds And Belt,带褶皱和皮带的短裙,Not available,S|M|L|XL,11.34,2026-01-20T07:22:20.888Z,2026-01-20T07:22:20.888Z
3,2936,Children,Sweaters,Camisola Infantil De Tricô Com Formas Geométricas,Kinderkrampf -Trikot Mit Geometrischen Formen,Jersey De Tricot Pour Enfants Avec Des Formes Géométriques,Jersey De Tejido Infantil Con Formas Geométricas,Children'S Knitting Jersey With Geometric Shapes,儿童编织球衣的几何形状,Not available,P|M|G,24.48,2026-01-20T07:22:20.888Z,2026-01-20T07:22:20.888Z
4,3139,Masculine,Underwear and Pajamas,Pijama Masculino Curto Com Camisa De Botão E Short Confortável,Kurzes Männliches Pyjama Mit Knopfhemd Und Bequemen Shorts,Pyjama Mâle Court Avec Chemise Boutonnée Et Shorts Confortables,Pijama Masculino Corto Con Camisa De Botones Y Pantalones Cortos Cómodos,Short Male Pajama With Button Shirt And Comfortable Shorts,穿着纽扣衬衫和舒适短裤的短男睡衣,Not available,M|L|XL|XXL,4.71,2026-01-20T07:22:20.888Z,2026-01-20T07:22:20.888Z
5,3683,Masculine,Suits and Blazers,Blazer Masculino De Veludo Slim Fit Com Forro De Cetim,Slim Fit Velvet Herren -Blazer Mit Satinfutter,Blazer Pour Hommes En Velours Slim Avec Doublure En Satin,Slim Fit Velvet Blazer Para Hombres Con Revestimiento De Satén,Slim Fit Velvet Men'S Blazer With Satin Lining,苗条适合天鹅绒男士西装外套和缎面衬里,Not available,M|L|XL|XXL,37.13,2026-01-20T07:22:20.888Z,2026-01-20T07:22:20.888Z


In [0]:
spark.read.table("retail_analytics.gold.dim_products").count()

17940